Dataflow Gen2 is suitable for static ingestion, but since we are designing a configuration-driven dynamic RDM framework with logic branching, a notebook provides the required programmability and scalability.”

🔹 when we use Notebook ?

whrn:

✔ Dynamic behaviour 
✔ Config read and then logic change
✔ For loop 
✔ Conditional logic apply 
✔ Conformed vs Mapping branch 
✔ _add detection build 

Notebook is programmable.

In [ ]:
# Step 0: We already loaded the config CSV into config_df.
# Now we will loop through each config row and sanity-check what we will ingest.

def print_config_summary(cfg):
    """
    Print a readable summary of one config row.
    This is just a verification step before we start reading SharePoint lists dynamically.
    """
    print("\n" + "-" * 60)
    print(f"Config Name     : {cfg['rdm_config_name']}")
    print(f"Entity          : {cfg['entity_name']}")
    print(f"SharePoint List : {cfg['sp_list_name']}")
    print(f"Target Table    : {cfg['silver_table_name']}")
    print(f"RDM Type        : {cfg['rdm_type']}")
    print(f"Include Columns : {cfg['include_columns']}")
    print(f"Business Keys   : {cfg['business_key_columns']}")
    print("-" * 60)


# Collect config rows from the Spark DataFrame
config_rows = config_df.collect()

# Print a summary for each row so we can confirm the config looks correct
for cfg in config_rows:
    print_config_summary(cfg.asDict())

In [ ]:
# Step 1: Prepare config rows for dynamic processing
# Convert comma-separated strings into Python lists so we can use them safely later.

prepared_configs = []

for cfg in config_df.collect():
    cfg_dict = cfg.asDict()

    include_cols = [c.strip() for c in cfg_dict["include_columns"].split(",") if c and c.strip()]
    business_keys = [c.strip() for c in cfg_dict["business_key_columns"].split(",") if c and c.strip()]

    prepared_configs.append({
        "config_name": cfg_dict["rdm_config_name"],
        "entity_name": cfg_dict["entity_name"],
        "sp_list_name": cfg_dict["sp_list_name"],
        "target_table": cfg_dict["silver_table_name"],
        "rdm_type": cfg_dict["rdm_type"],
        "include_columns": include_cols,
        "business_keys": business_keys
    })

# Human-readable check
for pc in prepared_configs:
    print("\n" + "-" * 60)
    print(f"Config Name   : {pc['config_name']}")
    print(f"Entity        : {pc['entity_name']}")
    print(f"SP List       : {pc['sp_list_name']}")
    print(f"Target Table  : {pc['target_table']}")
    print(f"RDM Type      : {pc['rdm_type']}")
    print(f"Include Cols  : {pc['include_columns']}")
    print(f"Business Keys : {pc['business_keys']}")